In [1]:
import pandas as pd
import torch
import numpy as np
from torchsummary import summary

# for embedding visualization later
import plotly.express as px 
import plotly.io as pio

# for VSCode plotly rendering
# pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.renderers.default = "plotly_mimetype+notebook"

pio.templates.default = "plotly_white"

from sklearn.model_selection import train_test_split

import spacy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
## miscellaneous data cleaning

start_episode = 20 # Start in Season 2, Season 1 is not very good
num_episodes = 50  # only pick this many episodes to train on

url = "https://github.com/PhilChodrow/PIC16B/blob/master/datasets/star_trek_scripts.json?raw=true"
star_trek_scripts = pd.read_json(url)

cleaned = star_trek_scripts["DS9"].str.replace("\n\n\n\n\n\nThe Deep Space Nine Transcripts -", "")
cleaned = cleaned.str.split("\n\n\n\n\n\n\n").str.get(-2)
text = "\n\n".join(cleaned[start_episode:(start_episode + num_episodes)])
for char in ['\xa0', 'à', 'é', "}", "{"]:
    text = text.replace(char, "")

In [6]:
WINDOW = 10 # predict next word from 10 previous words

predictors = []
targets    = []

for i in range(len(text) - WINDOW - 1):
    predictors.append(list(text[i:i+WINDOW]))
    # predictors.append(" ".join(word_seq[i:(i+WINDOW)]))
    targets.append(text[i+WINDOW])
len(predictors)
# i = 0
# len(tokenizer(predictors[0])), targets[i]


1570823

In [9]:
predictors[0:10]
targets[0:10]

['e', ' ', 'o', 'n', ' ', 'D', 'e', 'e', 'p', ' ']

In [10]:
from torch.utils.data import Dataset, DataLoader

class TextDataSet(Dataset):
    def __init__(self, predictors, targets):
        self.predictors = predictors
        self.targets = targets
    
    def __getitem__(self, index):
        return self.predictors[index], self.targets[index]

    def __len__(self):
        return len(self.targets)

data = TextDataSet(predictors, targets)

In [14]:
data[3]

(['a', 's', 't', '\n', 't', 'i', 'm', 'e', ' ', 'o'], 'n')